In [9]:
from __future__ import division
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

## Hyper-parameters

In [2]:
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

In [122]:
train_dataset = dsets.MNIST(root='../data/',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='../data/',
                           train=False,
                           transform=transforms.ToTensor())

Files already downloaded


In [123]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [6]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [7]:
rnn = RNN(input_size, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [139]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(custom_train_loader):
        images = Variable(images.view(-1, sequence_length, input_size))
        labels = Variable(labels)
        
        optimizer.zero_grad()
        outputs = rnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f'%(epoch+1, num_epochs, i+1, len(train_dataset)/batch_size, loss.data[0])

torch.save(rnn.state_dict(), 'rnn.pkl')

Epoch [1/2], Step [100/600], Loss: 0.5370
Epoch [1/2], Step [200/600], Loss: 0.5094
Epoch [1/2], Step [300/600], Loss: 0.2593
Epoch [1/2], Step [400/600], Loss: 0.1597
Epoch [1/2], Step [500/600], Loss: 0.0562
Epoch [1/2], Step [600/600], Loss: 0.3018
Epoch [1/2], Step [700/600], Loss: 0.4659
Epoch [1/2], Step [800/600], Loss: 0.3592
Epoch [1/2], Step [900/600], Loss: 0.3492
Epoch [1/2], Step [1000/600], Loss: 0.2787
Epoch [1/2], Step [1100/600], Loss: 0.2422
Epoch [1/2], Step [1200/600], Loss: 0.0766
Epoch [2/2], Step [100/600], Loss: 0.4040
Epoch [2/2], Step [200/600], Loss: 0.1881
Epoch [2/2], Step [300/600], Loss: 0.2738
Epoch [2/2], Step [400/600], Loss: 0.1237
Epoch [2/2], Step [500/600], Loss: 0.2097
Epoch [2/2], Step [600/600], Loss: 0.2284
Epoch [2/2], Step [700/600], Loss: 0.1121
Epoch [2/2], Step [800/600], Loss: 0.3286
Epoch [2/2], Step [900/600], Loss: 0.0938
Epoch [2/2], Step [1000/600], Loss: 0.3386
Epoch [2/2], Step [1100/600], Loss: 0.1221
Epoch [2/2], Step [1200/600],

In [17]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_length, input_size))
    outputs = rnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
print 'Test Accuracy of the model on the 10,000 test images: %d%%'%(100 * correct / total)

Test Accuracy of the model on the 10,000 test images: 97%


## Dataset building

In [133]:
import torch.utils.data as data
from mnist import MNIST
import numpy as np

mndata = MNIST('/Users/zal/CMU/Spring2017/16824/FinalProject/Data/mnist')
train_list = mndata.load_training()
test_list = mndata.load_testing()

In [134]:
train_np_data = np.array(train_list[0], dtype=float)
train_np_data = np.reshape(train_np_data, (60000, 28,28))
train_np_labels = np.array(train_list[1])

In [135]:
print train_np_data.shape, train_np_data.dtype
print train_np_labels.shape, train_np_labels.dtype

(60000, 28, 28) float64
(60000,) int64


In [136]:
train_data_samples = torch.from_numpy(train_np_data)
train_data_samples = train_data_samples.float()
train_data_labels = torch.from_numpy(train_np_labels)

In [137]:
print type(train_data_samples)
print train_data_samples.size()

<class 'torch.FloatTensor'>
torch.Size([60000, 28, 28])


In [138]:
custom_train_dataset = torch.utils.data.TensorDataset(train_data_samples, train_data_labels)
custom_train_loader = torch.utils.data.DataLoader(custom_train_dataset, batch_size=50, shuffle=True)

In [ ]:
def build_custom_dataset(data_path):
    # Load the data from disk
    mndata = MNIST(data_path)
    train_list = mndata.load_training()
    test_list = mndata.load_testing()
    
    # Load into numpy
    train_np_data = np.array(train_list[0])
    train_np_labels = np.array(train_list[1])
    
    # Load into Pytorch types
    train_data_samples = torch.from_numpy(train_np_data)
    train_data_samples = train_data_samples.float()      #Key-factor to train network
    train_data_labels = torch.from_numpy(train_np_labels)
    
    # build pytorch tensor dataset
    custom_train_dataset = torch.utils.data.TensorDataset(train_data_samples, train_data_labels)
    
    return custom_train_dataset